<a href="https://colab.research.google.com/github/Q8sh2ing/AFI-ForwardDeduplicate-Colab/blob/main/forward_dedup_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title setup
import os
clone_drive = False # @param {type:"boolean"}
if clone_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    # will still work without cloning.

if not os.path.exists ("/content/drive"):
    base_path = "/content/forward-dedup"
else:
    base_path = "/content/drive/MyDrive/forward-dedup"

inputdir = os.path.join(base_path, "input")
temp = ("/content/forward-dedup/frames")
outputdir = os.path.join(base_path, "output")

for path in [base_path, inputdir, temp, outputdir]:
    if not os.path.exists(path):
        os.makedirs(path)

from IPython.display import clear_output
clear_output()

%cd /content
if not os.path.exists ("/content/GMFSS_Fortuna"):
    !git clone https://github.com/98mxr/GMFSS_Fortuna.git
    %cd GMFSS_Fortuna

    requirements = """matplotlib
    moviepy
    numpy==1.21.0
    opencv-python
    rawpy
    scipy
    scikit-video
    scikit-image
    torch
    torchvision
    tqdm
    tensorboard
    lpips"""
    with open('/content/GMFSS_Fortuna/requirements.txt', 'w') as f:
        f.write(requirements)

    !pip install -r requirements.txt
    !git clone https://github.com/Q8sh2ing/AFI-ForwardDeduplicate-Colab.git
    %cd /content
    !gdown 157M4i1B9hjWs1K2AZVArSulkM9qV2sdH
    !unzip /content/AFI_ForwardDeduplicate_weights.zip -d /content/GMFSS_Fortuna/AFI-ForwardDeduplicate-Colab
    !rm /content/AFI_ForwardDeduplicate_weights.zip
    from IPython.display import clear_output
    clear_output()
    print("done.")

else:
    print("cloned.")

print("upload file at", inputdir)


In [ ]:
#@title inference
#@markdown TIMES (default 2)
TIMES = 2 # @param {type:"slider", min:2, max:16, step:1}
#@markdown number of max consistent deduplication counts - 1 (default 2)
NF = 2 # @param {type:"slider", min:1, max:8, step:1}
Model = "gmfss" # @param ["gmfss", "rife"]
#@markdown flow scale, generally use 1.0 with 1080p and 0.5 with 4K resolution for best result
SCALE = "1.0" # @param ["0.25", "0.5", "1.0", "2.0", "4.0"]
#@markdown clear output messeges
clear_omsg = True # @param {type:"boolean"}

import os
import cv2
import shutil
framepers = round((24000/1001) * TIMES, 3)
if Model=="rife":
    rnm = "RIFE"
else: rnm = "GMFSS"

file_list = [filename for filename in os.listdir(inputdir) if filename.endswith(('.mp4', '.mov', '.mkv', '.avi'))]
if not file_list:
    print(f"No files.\nPlease upload file at", inputdir)
else:
    total_files = len(file_list)
    for i, filename in enumerate(file_list):
        input_file = os.path.join(inputdir, filename)

        cap = cv2.VideoCapture(input_file)
        fps = cap.get(cv2.CAP_PROP_FPS)
        fps = round(fps, 3)
        in_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        in_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        cap.release()

        current_file_num = i + 1
        print(f"\tProcessing file {current_file_num} of {total_files}.")

        # normalizing to 23.976fps
        if fps < 23.976 or fps > 24:
            os.system(f'ffmpeg -loglevel error -y -i "{input_file}" -r 24000/1001 -c:v libx265 -x265-params profile=main10 -crf 5 -pix_fmt yuv420p -preset slow -c:a copy /content/normalized-temp.mov')
        else:
            os.system(f'ffmpeg -loglevel error -y -i "{input_file}" -c:v copy -c:a copy /content/normalized-temp.mov')

        # inference
        print(f"Running inference...")
        %cd /content/GMFSS_Fortuna/AFI-ForwardDeduplicate-Colab
        !python interpolate_video_forward.py -i /content/normalized-temp.mov -o {temp} -nf {NF} -t {TIMES} -m {Model} -scale {SCALE} -s -st 14 -stf

        # merge frames
        print("Merging frames...")
        os.system(f'ffmpeg -loglevel error -y -r {framepers} -i {temp}/%09d.png -i "{input_file}" -map 0:v -map 1:a? -vf scale={in_width}:{in_height} -crf 11 -pix_fmt yuv420p -preset slower -c:v h264 -profile:v main -c:a copy /content/interpolated-temp.mov')

        # clean up
        !rm -rf /content/forward-dedup/frames/* /content/forward-dedup/frames/scene
        !rm -rf /content/normalized-temp.mov
        # alternatively, you can cap fps to 60 without audio
        # !ffmpeg -loglevel error -r 60 -i {temp}/%09d.png -crf 10 -pix_fmt yuv420p -preset slower -c:v h264 -profile:v main -c:a copy /content/interpolated-temp.mov


        print(f"\t done {filename}.\n")


        dst_dir = outputdir
        old_file = [name for name in os.listdir(dst_dir) if name.endswith('.mov') and name.startswith(f'{filename}_AFI-{rnm}-{framepers}fps_')]
        if not old_file:
            num_files = 0
        else:
            num_files = max(int(file.split('_')[-1].split('.')[0]) for file in old_file)
        new_file_name = f'{filename}_AFI-{rnm}-{framepers}fps_{str(num_files + 1).zfill(3)}.mov'
        src_file = ("/content/interpolated-temp.mov")
        dst_file = os.path.join(dst_dir, new_file_name)
        shutil.move(src_file, dst_file)

        if clear_omsg:
            from IPython.display import clear_output
            clear_output()


    print (f"task done.")

In [ ]:
# remove outputfiles
!rm -r /content/forward-dedup/output/*
#!rm -r /content/forward-dedup/input/*